In [ ]:
#import pandas as pd

#df = pd.DataFrame()

#df = pd.read_csv('/content/AWCustomers.csv')

#('/content/drive')


In [ ]:
from google.colab import files

# Upload the file from your local machine
uploaded = files.upload()


Part I

In [ ]:

import pandas as pd
import numpy as np

df = pd.DataFrame()

df = pd.read_csv('AWCustomers.csv')

print(df.head())

# Display the summary statistics of each attribute
print(df.describe())

# Check the data types of each attribute
print(df.info())


#feature selection just be analysis

# Select relevant attributes (e.g., Age, Yearly Income, Commute Distance, etc.)
selected_features = ['AddressLine1', 'City', 'StateProvinceName', 'Occupation', 'Gender', 'HomeOwnerFlag', 'NumberCarsOwned', 'TotalChildren', 'YearlyIncome']
df_selected = df[selected_features]
print(df_selected.head())


# Data types for preprocessing
data_types = {
    'AddressLine1': 'Discrete, Nominal',
    'City': 'Discrete, Nominal',
    'StateProvinceName': 'Discrete, Nominal',
    'Occupation': 'Discrete, Nominal',
    'Gender': 'Discrete, Nominal',
    'HomeOwnerFlag': 'Discrete, Ratio',
    'NumberCarsOwned':'Discrete, Ratio',
    'TotalChildren': 'Discrete, Ratio',
    'YearlyIncome': 'Continuous, Ratio'
}

print("Data Types:")
for col, dtype in data_types.items():
    print(f"{col}: {dtype}")





Part II

In [ ]:

#handling null values


#Drop rows with missing values in the selected columns
#df_selected = df_selected.dropna()

#imputation for missing values in yearly income
from sklearn.impute import SimpleImputer

#Create the Imputer
median_imputer = SimpleImputer(missing_values=np.nan, strategy="median")
#Fit the imputer on the DataFrame and transform it to replace missing values with the median

#below is syntax for whole dataset
#df_imputed = pd.DataFrame(median_imputer.fit_transform(df_selected), columns=df_selected.columns)

#for some particular features
# Imputation for missing values in 'YearlyIncome'
# Imputation for missing values in 'YearlyIncome'
df_selected['YearlyIncome'] = median_imputer.fit_transform(df_selected['YearlyIncome'].values.reshape(-1, 1))
#median_imputer.fit_transform(df_selected['YearlyIncome'].values)

#mode imputer for rest of the features
mode_imputed_cols= ['AddressLine1', 'City', 'StateProvinceName', 'Occupation', 'Gender', 'HomeOwnerFlag', 'NumberCarsOwned', 'TotalChildren']
mode_imputer = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
df_selected[mode_imputed_cols] = mode_imputer.fit_transform(df_selected[mode_imputed_cols])

print("After Imputation:")
print(df_selected.head())


In [ ]:
import pandas as pd
from scipy.stats import zscore

# Ensure numeric columns are converted properly
df_selected['NumberCarsOwned'] = pd.to_numeric(df_selected['NumberCarsOwned'], errors='coerce')
df_selected['TotalChildren'] = pd.to_numeric(df_selected['TotalChildren'], errors='coerce')
df_selected['YearlyIncome'] = pd.to_numeric(df_selected['YearlyIncome'], errors='coerce')

# Apply z-score normalization
df_selected['NumberCarsOwned'] = zscore(df_selected['NumberCarsOwned'])
df_selected['TotalChildren'] = zscore(df_selected['TotalChildren'])
df_selected['YearlyIncome'] = zscore(df_selected['YearlyIncome'])

# Create dummy variables for categorical columns (e.g., 'Occupation', 'StateProvinceName')
dummies_occupation = pd.get_dummies(df_selected['Occupation'], drop_first=True)
dummies_state = pd.get_dummies(df_selected['StateProvinceName'], drop_first=True)

# Replace True/False with 1/0 in all dummy columns
dummies_occupation = dummies_occupation.astype(int)
dummies_state = dummies_state.astype(int)

# Concatenate the new dummy variables to the original DataFrame
df_selected = pd.concat([df_selected, dummies_occupation, dummies_state], axis=1)

# Optionally, drop the original categorical columns if they're no longer needed
df_selected = df_selected.drop(columns=['Occupation', 'StateProvinceName'])

# Print the first few rows of the dummy variables to verify them
print("Occupation dummies:")
print(dummies_occupation.head())

print("StateProvinceName dummies:")
print(dummies_state.head())

# Display the first few rows after normalization and encoding
print("After normalization and encoding:")
print(df_selected.head())


In [ ]:
#Binning

import pandas as pd
import numpy as np

# Example of binning (discretization) on YearlyIncome (a continuous attribute)
# Discretizing YearlyIncome into 5 bins (you can change the number of bins)
df_selected['YearlyIncome_binned'] = pd.cut(df_selected['YearlyIncome'], bins=5, labels=False)

# If you prefer binning with custom bin edges, you can specify them manually:
# custom_bins = [0, 30000, 60000, 90000, 120000, np.inf]
# df_selected['YearlyIncome_binned'] = pd.cut(df_selected['YearlyIncome'], bins=custom_bins, labels=False)

# Checking the binned YearlyIncome
print("Binned YearlyIncome:")
print(df_selected[['YearlyIncome', 'YearlyIncome_binned']].head())

# Other continuous attributes that may need binning (if they are not one-hot encoded):
# If there are other continuous features you want to bin, apply pd.cut similarly.
# Example: Binning TotalChildren into 3 bins
df_selected['TotalChildren_binned'] = pd.cut(df_selected['TotalChildren'], bins=3, labels=False)

# Binning NumberCarsOwned into 3 bins (if needed)
df_selected['NumberCarsOwned_binned'] = pd.cut(df_selected['NumberCarsOwned'], bins=3, labels=False)

# Check the discretized attributes
print("Binned attributes:")
print(df_selected[['TotalChildren', 'TotalChildren_binned', 'NumberCarsOwned', 'NumberCarsOwned_binned']].head())

# Drop the original columns if you no longer need them after binning
df_selected = df_selected.drop(columns=['YearlyIncome', 'TotalChildren', 'NumberCarsOwned'])

# Final dataframe with discretized features
print("Final DataFrame after discretization:")
print(df_selected.head())



Part III

In [ ]:

from sklearn.metrics import jaccard_score

# Assuming row1 and row2 are extracted from the DataFrame
row1 = df_selected.iloc[0]
row2 = df_selected.iloc[1]

# Convert the rows to numeric to ensure they are binary or numeric
row1 = pd.to_numeric(row1, errors='coerce')
row2 = pd.to_numeric(row2, errors='coerce')

# Ensure no NaN values (in case of coercion issues)
row1 = row1.fillna(0).astype(int)
row2 = row2.fillna(0).astype(int)

# Now calculate Jaccard Similarity for multiclass
jaccard_sim = jaccard_score(row1, row2, average='macro')  # You can also use 'micro' or 'weighted'

print(f"Jaccard Similarity: {jaccard_sim}")



In [ ]:

# Assuming Commute Distance is already ordinally encoded
correlation = np.corrcoef(df_selected['CommuteDistance'].astype(float), df_selected['YearlyIncome'])[0, 1]
print("Correlation between Commute Distance and Yearly Income:", correlation) #our dataset doesn't have commute dist
